In [1]:
import os
import ast
import pandas as pd
import numpy as np
import copy
from itertools import islice

In [2]:
#Return first n items of the iterable as a list
def take(n, iterable):
    return list(islice(iterable, n))

In [3]:
import logging

# create logger
logger = logging.getLogger(__name__)
# set log level for all handlers to debug
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
# best for development or debugging
consoleHandler = logging.StreamHandler()
consoleHandler.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
consoleHandler.setFormatter(formatter)

# add ch to logger
logger.addHandler(consoleHandler)


In [16]:
pap_auth_df = pd.read_csv("papId_authId.csv", dtype='str')
pap_auth_df.head()

,Paper Id,Author Id
0,9112005,['1']
1,9112014,"['2', '34', '30', '33']"
2,9109003,['3']
3,9108020,"['4', '5']"
4,9112074,"['6', '7', '11', '12']"


In [18]:
single_author_word = pd.read_csv("single_author_topic_words.csv", dtype='str')
paper_topic_word = pd.read_csv("multi_authored_papers_topic_model.csv", dtype='str')

In [19]:
paper_topic_word.head()

,Paper Id,Topic Words
0,0001001,"{'space': 0.06532662655638953, 'string': 0.053..."
1,0001002,"{'domain': 0.13554957637373383, 'wall': 0.1255..."
2,0001003,"{'gauge': 0.3181979025545328, 'eipx': 0.172512..."
3,0001004,"{'mirror': 0.08901499116017353, 'energy': 0.08..."
4,0001005,"{'chiral': 0.03881194003739911, 'algebra': 0.0..."


In [20]:
single_author_word.head()

,Author Id,Topic Words
0,1,"{'string': 0.09652447228585227, 'brane': 0.089..."
1,3,"{'fields': 0.07744539996852072, 'chiral': 0.04..."
2,10,"{'quantum': 0.033096638147438726, 'potential':..."
3,14,"{'theory': 0.13321210713137357, 'loop': 0.1209..."
4,22,"{'bravais': 0.09412688548146708, 'classes': 0...."


In [6]:
single_author_word = single_author_word.set_index("Author Id").to_dict()["Topic Words"]
paper_topic_word = paper_topic_word.set_index("Paper Id").to_dict()["Topic Words"]
pap_auth_df = pap_auth_df.set_index("Paper Id").to_dict()["Author Id"]

In [7]:
len(single_author_word) # Number of authors who have published papers on their own

4187

In [8]:
authors_words = copy.deepcopy(single_author_word)
paper_unknown_list = {}
paper_papWords = {}
count = 0

for it in range(0, 75):
    for paper_id, author_id_list in pap_auth_df.items():
        rowAuth = ast.literal_eval(author_id_list)
        if paper_id not in paper_topic_word.keys():
            continue
        count += 1
        if len(rowAuth) > 1:
            unknown_list = []
            paper_words = ast.literal_eval(paper_topic_word[paper_id])
            #logger.info(type(paper_words))
            #break
            for authId in rowAuth:
                if authId in authors_words.keys():
                    paper_words = {k:v for k,v in paper_words.items() if k not in authors_words[authId]}
                else:
                    unknown_list.append(authId)
            if len(unknown_list) == 1:
                authors_words[unknown_list[0]] = paper_words
            paper_papWords[paper_id] = paper_words
            paper_unknown_list[paper_id] = unknown_list
    #break
for paper_id, uk_list in paper_unknown_list.items():
    paper_words = paper_papWords[paper_id]
    #break
    for authId in uk_list:
        if authId not in authors_words.keys():
            authors_words[authId] = paper_words

In [9]:
len(set(authors_words) - set(single_author_word)) # Difference between new dict and single author dict

8649

In [10]:
len(authors_words) # Number of authors who have topic words

12836

In [11]:
d = authors_words
d = {key:val for key, val in d.items() if val != {}}

In [13]:
#Creating and downloading dataframe with author id and topic words for both multi and single authors
author_words_df = pd.DataFrame(d.items(),columns = ['Author Id','Topic Words'])
author_words_df.describe()

,Author Id,Topic Words
count,12836,12836
unique,12836,11386
top,4632,"{'phase': 0.03929639914990857, 'transition': 0..."
freq,1,15


In [21]:
author_words_df.head()

,Author Id,Topic Words
0,1,"{'string': 0.09652447228585227, 'brane': 0.089..."
1,3,"{'fields': 0.07744539996852072, 'chiral': 0.04..."
2,10,"{'quantum': 0.033096638147438726, 'potential':..."
3,14,"{'theory': 0.13321210713137357, 'loop': 0.1209..."
4,22,"{'bravais': 0.09412688548146708, 'classes': 0...."


In [15]:
author_words_df.to_csv('author_topic_words.csv', index=False) # Author Id, Top 30 words for that author

50 times -> 10203th row
75 times -> 10203th row